In [ ]:
import ezomero
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import sys
import os
from src.omero_annotation import create_collection, link_collection_to_image, add_node_annotation

import numpy as np
from omero.gateway import BlitzGateway
from omero.model import DatasetI, ImageI
from omero.rtypes import rstring, rint


# Load environment variables from .env file
load_dotenv(override = True)

from micro_sam.automatic_segmentation import get_predictor_and_segmenter, automatic_instance_segmentation
predictor, segmenter = get_predictor_and_segmenter(
    model_type="vit_b_lm",  # You can also use 'vit_b', 'vit_l', 'vit_h', 'vit_b_256' or your custom model checkpoint path.
    checkpoint=None,  # Replace this with your custom checkpoint.
    is_tiled=False,  # Switch to 'True' in case you would like to perform tiling-window based prediction.
)

In [ ]:
def post_image(conn, img_array, image_name, dataset_id):
    # img_array should be 5D: (T, Z, C, Y, X)
    size_t, size_z, size_c, size_y, size_x = img_array.shape
    pixels = []
    for t in range(size_t):
        for z in range(size_z):
            for c in range(size_c):
                plane = img_array[t, z, c, :, :]
                pixels.append(plane)
    # Create image (pass an iterator, not a list)
    image = conn.createImageFromNumpySeq(
        iter(pixels),  # <-- fix here
        image_name,
        sizeZ=size_z,
        sizeC=size_c,
        sizeT=size_t,
        dataset=conn.getObject("Dataset", dataset_id),
        channelList=None,
        description="Segmentation mask"
    )
    return image.getId()


In [2]:
conn = ezomero.connect(
    os.getenv("OMERO_USERNAME"), 
    os.getenv("OMERO_PASSWORD"), 
    os.getenv("OMERO_GROUP"),
    host=os.getenv("OMERO_HOST"),
    port=int(os.getenv("OMERO_PORT")),
    secure=os.getenv("OMERO_SECURE") == "True"
)

In [3]:
dataset_id = 4272  # Replace with your dataset ID
im_ids = ezomero.get_image_ids(conn, dataset=dataset_id)
im_ids

[35478, 35480, 35479]

In [5]:
# Get images from OMERO with ezomero
segmentation = []
for id in im_ids:
    img,img_array = ezomero.get_image(conn, id)

    segmentation.append(automatic_instance_segmentation(
        predictor=predictor,
        segmenter=segmenter,
        input_path=img_array[0,0,:,:,0],
        ndim=2,
        tile_shape=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a tile shape
        halo=None,  # If you set 'is_tiled' in 'get_predictor_and_segmeter' to True, set a halo shape.
    ))


Initialize instance segmentation with decoder: 100%|██████████| 1/1 [00:03<00:00,  3.59s/it]


In [ ]:

# Upload the segmentation masks
for i, seg in enumerate(segmentation):
    coll_id = create_collection(conn, "micro_sam_segmentation")
    source_id = im_ids[i]
    link_collection_to_image(conn, coll_id, source_id)
    add_node_annotation(conn, source_id, "intensities", coll_id, node_name="raw")
    
    # Convert 2D (Y,X) to 5D (T,Z,C,Y,X) with single planes/channels/timepoint
    seg_5d = seg[np.newaxis, np.newaxis, np.newaxis, :, :]
    seg_id = post_image(conn, seg_5d, image_name=f"segmentation_{im_ids[i]}", dataset_id=dataset_id)
    link_collection_to_image(conn, coll_id, seg_id)
    add_node_annotation(conn, seg_id, "labels", coll_id, node_name="micro_sam_segmentation")

In [20]:
map_ann_to_delete = ezomero.get_map_annotation_ids(conn, object_type="Image", object_id=35480)
conn.deleteObjects('Annotation', map_ann_to_delete, wait=True)  

c45c27d4-084a-4775-b55e-9f2730c50a49/IHandle2d6ca17c-97fc-4d28-bbf2-bca40cdf603d -t -e 1.1:tcp -h 128.176.233.5 -p 44685 -t 60000

# Try to append to exisiting collection